In [19]:
from icalendar import Calendar, Event
from datetime import datetime
from pytz import UTC # timezone
import pandas as pd
from openpyxl import load_workbook
import recurring_ical_events
datetime.today().strftime('%Y-%m-%d')
import random
from datetime import date
import datetime
import win32com.client
import win32com.client as win32
import datetime
import os
SettingWithCopyWarning = None
TODAY = date.today().strftime('%Y-%m-%d')
TEMPLATE_PATH = f'C:/Users/mariu/Desktop/tutor-invoice-generator/sablonas.xlsx'
TEMPORARY_PATH =  f'C:/Users/mariu/Desktop/tutor-invoice-generator/temporary.xlsx'

In [2]:
with open('calendar.ics','rb') as g:
    gcal = Calendar.from_ical(g.read())

In [3]:
events = [dict(event) for event in recurring_ical_events.of(gcal).all()]

In [4]:
df = pd.DataFrame(events)

In [5]:
def to_date(x):
    return x.dt.strftime('%Y-%m-%d-%H:%M')

In [6]:
df['DTSTART'] = df['DTSTART'].apply(to_date)
df['DTEND'] = df['DTEND'].apply(to_date)

In [7]:
df = df[(df['DTSTART'] >= '2022-09-01') & (df['DTSTART'] < '2022-10-01')]

In [8]:
df = df[['SUMMARY', 'DESCRIPTION', 'DTSTART', 'DTEND']]

In [9]:
df['DTEND'] = pd.to_datetime(df['DTEND'])
df['DTSTART'] = pd.to_datetime(df['DTSTART'])

In [10]:
df['DURATION'] = (df['DTEND'] - df['DTSTART']).apply(pd.Timedelta.total_seconds)/3600

In [11]:
df = df[df['SUMMARY'].apply(lambda x: '(p)' in x)]

In [12]:
def get_last_month():
    today = datetime.date.today()
    first = today.replace(day=1)
    last_month = first - datetime.timedelta(days=1)
    return last_month.strftime("%Y/%m")

In [23]:
def prep_invoice(client_name, client_email, amount, rate=17):
    #load excel file
    workbook = load_workbook(filename=TEMPLATE_PATH)
    #open workbook
    sheet = workbook.active
 
    #modify the desired cell
    sheet['D3'] = f'Serija {date.today().strftime("%Y")} Nr. {random.randint(1000,10000)}'
    sheet["E6"] = date.today().strftime('%Y-%m-%d')
    sheet["B21"] = client_name
    sheet["B22"] = client_email
    sheet["B29"] = f'Korepetitoriaus paslaugos ({amount} val.)'
    sheet["D29"] = get_last_month()
    sheet['F29'] = amount * rate
    sheet['F31'] = amount * rate
    sheet['D42'] = client_name

    
    try:
        os.remove(TEMPORARY_PATH)
    except OSError:
        pass
    #save the file
    workbook.save(filename=TEMPORARY_PATH)
    workbook.close()

In [24]:
def print_pdf(pdf_name):
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False
    wb = o.Workbooks.Open(TEMPORARY_PATH)
    ws_index_list = [1] #say you want to print these sheets
    path_to_pdf = f'C:/Users/mariu/Desktop/tutor-invoice-generator/invoices/{pdf_name}.pdf'
    wb.WorkSheets(ws_index_list).Select()
    wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
    wb.Close()
    return path_to_pdf

In [25]:
def send_invoice(client_email, path_to_pdf):
    olApp = win32.Dispatch('Outlook.Application')
    olNS = olApp.GetNameSpace('MAPI')
    # construct email item object
    mailItem = olApp.CreateItem(0)
    mailItem.Subject = 'Sąskaita už korepetitoriaus paslaugas'
    mailItem.BodyFormat = 1
    mailItem.Body = f'''
    Sveiki,

    Prisegu sąskaitą faktūrą už praeitą mėnėsį teiktas korepetitoriaus paslaugas.

    Pagarbiai,
     Marius Arlauskas

    *** Dėmėsio ši sąskaita buvo sugeneruota automatiškai, patikrinkite ar nėra neatitikčių.
    *** Kilus problemai rašykite marius.arlauskas01@gmail.com
    '''
    mailItem.To = client_email
    mailItem.Sensitivity  = 2
    mailItem.Attachments.Add(Source=path_to_pdf)
    mailItem._oleobj_.Invoke(*(64209, 0, 8, 0, olNS.Accounts.Item('marius.arlauskas01.spam@gmail.com')))
    mailItem.Display()
    mailItem.Save()
    mailItem.Send()

In [26]:
def handle_invoice(client_name, client_email, amount, charge=17):
    prep_invoice(client_name, client_email, amount, charge)
    path_to_pdf = print_pdf(f'{client_email}_{TODAY}')
    send_invoice(client_email, path_to_pdf)

In [27]:
df.groupby('SUMMARY').sum()

,DURATION
SUMMARY,
Gabija 12kl. (p),3.0
Gabrielius 9kl. (p),4.0
Gintė 7kl. (p),3.0
Ieva 7kl. (p),1.0
Rusnė 9kl. (p),4.5
Rusnė 9kl.(p),1.5


In [28]:
clients = {
    'Gabrielius 9kl. (p)': {
        'client_name' : 'Gabrielius Pūkys',
        'client_email': 'svietlanele@gmail.com',
        'charge': 17
    },
    'Dominykas 10kl. (p)': {
        'client_name' : 'Dominykas Borsak',
        'client_email': 'borsak.jurate@gmail.com',
        'charge': 17
    },
    'Rusnė 9kl. (p)': {
        'client_name' : 'Rusnė Kupliauskaitė',
        'client_email': 'r.kupliauskas@gmail.com',
        'charge': 17
    },
    'Akvilė 11kl. (p)': {
        'client_name' : 'Akvilė Pažemeckaitė',
        'client_email': 'meljolaa@gmail.com',
        'charge': 17
    },
    'Gabija 12kl. (p)': {
        'client_name' : 'Gabija Rutkauskaitė',
        'client_email': 'gabyte2004@gmail.com',
        'charge': 17
    },
    
}